# Loop 69 Analysis: Submission Error Investigation

The submission failed with "Evaluation metric raised an unexpected error". Let me investigate the issue.

In [1]:
import pandas as pd
import numpy as np
import json

# Load session state
with open('/home/code/session_state.json', 'r') as f:
    state = json.load(f)

# Check submission history
submissions = state.get('submissions', [])
print(f'Total submissions: {len(submissions)}')
print('\nSubmission history:')
for s in submissions:
    print(f"  {s.get('experiment_id', 'N/A')}: CV={s.get('cv_score', 'N/A')}, LB={s.get('lb_score', 'N/A')}")

Total submissions: 22

Submission history:
  exp_000: CV=0.011081, LB=0.09816
  exp_001: CV=0.012297, LB=0.10649
  exp_003: CV=0.010501, LB=0.09719
  exp_005: CV=0.01043, LB=0.09691
  exp_006: CV=0.009749, LB=0.09457
  exp_007: CV=0.009262, LB=0.09316
  exp_009: CV=0.009192, LB=0.09364
  exp_012: CV=0.009004, LB=0.09134
  exp_024: CV=0.008689, LB=0.08929
  exp_026: CV=0.008465, LB=0.08875
  exp_030: CV=0.008298, LB=0.08772
  exp_035: CV=0.009825, LB=0.09696
  exp_049: CV=0.008092, LB=
  exp_050: CV=0.008092, LB=
  exp_052: CV=0.01088, LB=
  exp_053: CV=0.008092, LB=
  exp_054: CV=0.008504, LB=
  exp_055: CV=0.008504, LB=
  exp_057: CV=0.009263, LB=
  exp_063: CV=0.011171, LB=
  exp_064: CV=0.009227, LB=
  exp_065: CV=0.008811, LB=


In [2]:
# Analyze CV-LB relationship from successful submissions
successful = [(0.0111, 0.0982), (0.0123, 0.1065), (0.0105, 0.0972), (0.0104, 0.0969),
              (0.0097, 0.0946), (0.0093, 0.0932), (0.0092, 0.0936), (0.0090, 0.0913),
              (0.0087, 0.0893), (0.0085, 0.0887), (0.0083, 0.0877), (0.0098, 0.0970)]

cv_scores = [x[0] for x in successful]
lb_scores = [x[1] for x in successful]

from sklearn.linear_model import LinearRegression
X = np.array(cv_scores).reshape(-1, 1)
y = np.array(lb_scores)
reg = LinearRegression().fit(X, y)

print(f'CV-LB Linear Fit: LB = {reg.coef_[0]:.2f} * CV + {reg.intercept_:.4f}')
print(f'R-squared = {reg.score(X, y):.4f}')
print(f'\nIntercept: {reg.intercept_:.4f}')
print(f'Target: 0.0347')
print(f'\nTo reach target LB 0.0347:')
required_cv = (0.0347 - reg.intercept_) / reg.coef_[0]
print(f'Required CV = (0.0347 - {reg.intercept_:.4f}) / {reg.coef_[0]:.2f} = {required_cv:.4f}')

CV-LB Linear Fit: LB = 4.31 * CV + 0.0525
R-squared = 0.9505

Intercept: 0.0525
Target: 0.0347

To reach target LB 0.0347:
Required CV = (0.0347 - 0.0525) / 4.31 = -0.0041


In [3]:
# Check the submission file format
sub = pd.read_csv('/home/submission/submission.csv')
print('Submission shape:', sub.shape)
print('Columns:', sub.columns.tolist())
print('\nTask 0 (single solvent):')
print(f'  Folds: {sorted(sub[sub["task"]==0]["fold"].unique())}')
print(f'  Total rows: {len(sub[sub["task"]==0])}')
print('\nTask 1 (full data):')
print(f'  Folds: {sorted(sub[sub["task"]==1]["fold"].unique())}')
print(f'  Total rows: {len(sub[sub["task"]==1])}')

Submission shape: (1883, 8)
Columns: ['id', 'index', 'task', 'fold', 'row', 'target_1', 'target_2', 'target_3']

Task 0 (single solvent):
  Folds: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
  Total rows: 656

Task 1 (full data):
  Folds: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
  Total rows: 1227


In [4]:
# Check for any issues with the predictions
print('Target value ranges:')
for col in ['target_1', 'target_2', 'target_3']:
    print(f'  {col}: [{sub[col].min():.6f}, {sub[col].max():.6f}]')

print('\nAny NaN:', sub.isna().sum().sum())
print('Any Inf:', (sub.select_dtypes(include='number').abs() == float('inf')).sum().sum())
print('Any negative:', (sub[['target_1', 'target_2', 'target_3']] < 0).sum().sum())
print('Any > 1:', (sub[['target_1', 'target_2', 'target_3']] > 1).sum().sum())

Target value ranges:
  target_1: [0.000000, 0.412037]
  target_2: [0.000000, 0.414972]
  target_3: [0.000000, 0.995417]

Any NaN: 0
Any Inf: 0
Any negative: 0
Any > 1: 0


In [5]:
# The submission format looks correct. The error might be from Kaggle's evaluation system.
# Let me check if there's a pattern in the failed submissions.

# Failed submissions: exp_049, exp_050, exp_052, exp_053, exp_054, exp_055, exp_057, exp_063, exp_064, exp_065
# All of these had "Evaluation metric raised an unexpected error"

# Successful submissions: exp_000 through exp_035 (except some)
# The last successful submission was exp_035 with LB 0.0970

# What changed between exp_035 and exp_049?
# - exp_035 was a simple MLP with Spange features
# - exp_049+ were more complex ensembles

print('Analysis: The submission format is correct.')
print('The error is likely from Kaggle\'s evaluation system, not our submission format.')
print('\nPossible causes:')
print('1. Kaggle evaluation system issue (temporary)')
print('2. Notebook structure issue (extra cells after final cell)')
print('3. Model definition issue (complex ensemble not compatible)')
print('\nRecommendation: Try a simpler model that matches the template exactly.')

Analysis: The submission format is correct.
The error is likely from Kaggle's evaluation system, not our submission format.

Possible causes:
1. Kaggle evaluation system issue (temporary)
2. Notebook structure issue (extra cells after final cell)
3. Model definition issue (complex ensemble not compatible)

Recommendation: Try a simpler model that matches the template exactly.
